In [150]:
import pandas as pd
import os,time,tqdm

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait,Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, WebDriverException, TimeoutException

import requests
from bs4 import BeautifulSoup

In [2]:
os.chdir('./data')

In [3]:
# 판매실적 데이터에서 모델아이디를 추출 (소형SUV만)
data = pd.read_csv('monthly_utf.csv')
model_ids = [str(el) for el in list(data[data.seg=='RU2'].model.unique())]
del data

In [5]:
os.chdir('./..')

path = './chromedriver.exe'

options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('disable-gpu')

driver = webdriver.Chrome(path,options=options)

In [184]:
# driver.quit()

In [16]:
# utilities
class Util:
    @classmethod
    def make_url(cls,params):
            url = 'http://m.auto.danawa.com/auto/?Work=model'
            for k,v in params.items():
                if v is not None:
                    url += '&'+k+'='+v
            return url

세부 모듈 정의

- 가격 탭
- 제원 탭
- 사양/옵션 탭

In [ ]:
#종합정보 탭 -> car_base 테이블
# def get_main():
#     params['Tab'] = 'main'
#     url_main = Util.make_url(params)
#     driver.get(url_main)

#     # 브랜드id nm,  세그id,nm 모델id, nm, 출시, 

In [ ]:
# 가격 탭 -> 라인업 가져오기
def get_lineup():
    params['Tab'] = 'price'
    url_price = Util.make_url(params)
    driver.get(url_price)
    
    try:
    # 라인업쭉저장(최근년도만 직접보며 라인업 선별, id,nm,출시,  )
        buttonSelect = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable(
                            (By.ID, "buttonSelect")))
        buttonSelect.click()

        popup_data = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.ID, "popup_data")))
    except TimeoutException as te: # 아예 라인업 없는 경우
            return []
            
    lineup_lis = popup_data.find_element_by_class_name('trimInfo').find_elements_by_tag_name('li')
    
    lu_len = len(lineup_lis) # stale Element Error 때문에 우선 라인업 길이만 가져옴
    
    driver.get(url_price)
    lineups = []
    for lu_idx in range(lu_len):
        driver.get(url_price)
        buttonSelect = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable(
                        (By.ID, "buttonSelect")))
        buttonSelect.click()
    
        popup_data = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.ID, "popup_data")))
        
        lu_buttons = popup_data.find_element_by_class_name('trimInfo').find_elements_by_tag_name('li')
        lu_start = lu_buttons[lu_idx].find_element_by_tag_name('a').get_attribute('href').find('&Lineup=')
        lu_cd = lu_buttons[lu_idx].find_element_by_tag_name('a').get_attribute('href')[lu_start+8:lu_start+13]
        lu_nm = lu_buttons[lu_idx].find_element_by_tag_name('a').find_element_by_tag_name('span').text.strip()
        lineups.append((lu_cd,lu_nm))
    return lineups

In [ ]:
# 가격 탭
def get_price():
    params['Tab'] = 'price'
    url_price = Util.make_url(params)
    driver.get(url_price)
    
    price_list = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.ID, "priceList")))
    dts = price_list.find_elements_by_tag_name('dt')
    trim_prices = []
    for dt in dts:
        try:
            trim_prices.append((dt.find_element_by_class_name('title').text.strip(), dt.find_element_by_class_name('price').text.strip()))
        except NoSuchElementException as nse: # '공통사양' 인 element
            continue
            
    return trim_prices # trim 개수와 트림,가격 리스트 리턴


In [ ]:
#  제원 탭
def get_spec()->dict:
    global spec_valid
    
    params['Tab'] = 'spec'
    url_spec = Util.make_url(params)
    
    ## 제원 테이블 request -> 트림인덱스맞게 tdTitle: tdCenter 끼워넣기
    res = requests.get(url_spec,headers=headers)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text,'lxml')
    try: 
        table = soup.find('table',{'class':'tableBlock'})
        tr_list = table.find_all('tr')
    except AttributeError as ae: # 제원 탭 아예 존재 안함
        spec_valid = False
        return
    
    spec = {}
    for tr in tr_list:
        spec[tr.find('td').text.strip()] = [td.text.strip() for td in tr.find_all('td',{'class':'tdCenter'})]
    
    return spec
    

In [ ]:
# 사양/옵션 탭
def get_opt()->dict:
    global opt_valid

    params['Tab'] = 'item'
    url_item = Util.make_url(params)
    
    ## item 테이블 request -> 트림인덱스맞게 tdTitle: tdCenter 끼워넣기
    res = requests.get(url_item,headers=headers)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text,'lxml')
    
    try:
        table = soup.find('table',{'class':'tableBlock'})
        tr_list = table.find_all('tr')
    except AttributeError as ae: # 사양/옵션 탭 아예 존재 안함
        opt_valid = False
        return
    
    item = {}
    for tr in tr_list:
        item[tr.find('td').text.strip()] = [td.text.strip() for td in tr.find_all('td',{'class':'tdCenter'})]
    
    return item

메인

In [182]:
if __name__=='__main__':
    params = {} # url 의 params
    params['Model'] = None # 모델 중심으로 데이터 가져오겠다
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/... Safari/537.36'}
    
    car_detail_base = {'model':[],'lu_cd':[],'lu_nm':[],'trim':[],'trim_idx':[],'price':[],'specs':[],'opt':[]}
    car_detail = pd.DataFrame(car_detail_base) # pandas DF 형태의 스키마
    
    unhandled_e = [] # 전체적인 예외처리 위함
    for model_id in tqdm.tqdm(model_ids): # 판매실적있는 model에 대해서만 진행
        try:
            params['Model'] = str(model_id)

            lineups = get_lineup() # 라인업 쭉 가져온다           
            for lineup in lineups: # 라인업에 대해 진행
                params['LineUp'] = str(lineup[0])
                
                spec_valid =True # 제원, 옵션 탭 존재 여부
                opt_valid = True

                trim_prices = get_price() # 트림과 가격 정보 가져옴
                trim_len = len(trim_prices) # 트림 개수만큼 데이터 쌓을 것
                spec_dic = get_spec() # 스펙 정보 딕셔너리 형태로 가져옴
                opt_dic = get_opt() # 옵션 정보 딕셔너리 형태로 가져옴

                tmp_detail_base = {'model':[],'lu_cd':[],'lu_nm':[],'trim':[],'trim_idx':[],'price':[],'specs':[],'opt':[]}
                for trim_idx in range(trim_len): # 트림에 대해 tmp_detail_base를 채워가겠다
                    tmp_detail_base['model'].append(params['Model'])
                    tmp_detail_base['lu_cd'].append(params['LineUp'])
                    tmp_detail_base['lu_nm'].append(lineup[1])
                    tmp_detail_base['trim'].append(trim_prices[trim_idx][0])
                    tmp_detail_base['trim_idx'].append(trim_idx)
                    tmp_detail_base['price'].append(trim_prices[trim_idx][1])

                    tmp_spec_dic = {}
                    if spec_valid:
                        for k,v in spec_dic.items():
                            tmp_spec_dic[k] = v[trim_idx]
                    tmp_detail_base['specs'].append(tmp_spec_dic)

                    tmp_opt_dic = {}
                    if opt_valid:
                        for k,v in opt_dic.items():
                            tmp_opt_dic[k] = v[trim_idx]
                    tmp_detail_base['opt'].append(tmp_opt_dic)


                tmp_detail = pd.DataFrame(tmp_detail_base)
                car_detail = pd.concat((car_detail,tmp_detail), axis=0,sort=False) # tmp_detail을 기존 car_detail에 쌓기
        
        except Exception as e: # 예외 발생시 모델id 저장해 다시 처리하자
            print(model_id, e)
            unhandled_e.append(model_id)
            continue
                
            


In [171]:
car_detail.price = car_detail.price.str.replace(',','').astype('int64')
car_detail.trim_idx = car_detail.trim_idx.astype('int32')

In [183]:
car_detail.sample(3)

,model,lu_cd,lu_nm,trim,trim_idx,price,specs,opt
99,2146,36639,2WD (1월 기준),CLUBBY PARK A/T,5,25000000,"{'치수': '4,410', '전폭(㎜)': '1,830', '전고(㎜)': '1,...","{'안전': '운전석, 동반석', '주행안전': 'EBD-ABS, ESP & BAS..."
354,3105,41102,,RX 최고급형 A/T,4,26720000,"{'치수': '4,410', '전폭(㎜)': '1,830', '전고(㎜)': '1,...","{'안전': '운전석, 동반석, 사이드(앞), 커튼(앞뒤)', '주행안전': 'AB..."
184,3054,40175,디젤,LT A/T,2,21050000,"{'치수': '4,245', '전폭(㎜)': '1,775', '전고(㎜)': '1,...","{'안전': '운전석, 동반석, 사이드(앞), 커튼(앞뒤)', '주행안전': 'AB..."


In [178]:
car_detail.reset_index(drop=True,inplace=True)

In [179]:
car_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 8 columns):
model       937 non-null object
lu_cd       937 non-null object
lu_nm       937 non-null object
trim        937 non-null object
trim_idx    937 non-null int32
price       937 non-null int64
specs       937 non-null object
opt         937 non-null object
dtypes: int32(1), int64(1), object(6)
memory usage: 55.0+ KB


In [180]:
os.chdir('./data')
car_detail.to_csv('car_detail_190207.csv',encoding='utf-8') # csv형태로 저장 완료